### Лекция 4. Классы продолжение

<br />

##### Спецификация вызова конструктора базового класса

Базовый класс может иметь несколько конструкторов, а наследник может выбирать, какой именно из конструкторов он хочет вызвать. Это определяется в списке инициализации конструктора

```c++
class Animal
{
public:
    Animal();
    Animal(const std::string& a_name);
    Animal(const std::string& a_name, unsigned a_age);

private:
    std::string name;
    unsigned age;
};

class Turtle : public Animal
{
public:
    // неявно вызывает Animal()
    Turtle() {}    
    // эквивалентно записи:
    // Turtle() : Animal() {}
    
    // явный вызов конкретного конструктора Animal
    Turtle(const std::string& name)
        : Animal(name)  // сначала вызываем конструктор базового класса
        , color(White)  // затем конструкторы членов в порядке их определения
    {
    }
    
private:
    Color color = Black;  // значение для инициализации по умолчанию
};
```

<br />

##### Виртуальные функции

Задача, которую решает мезанизм виртуальных функций: класс-наследник может доопеределять / уточнять поведение базового класса

Примеры:

```c++
class Animal
{
public:
    virtual void cry() {}
};

class Dog : public Animal
{
public:
    virtual void cry() {
        std::cout << "augh!" << std::endl;
    }
};

class Cat : public Animal
{
public:
    virtual void cry() {
        std::cout << "meow!" << std::endl;
    }
};
```

__Вопрос знатокам__:

<details>
<summary>Какая ошибка здесь допущена?</summary>
<p>
    
Нет виртуального деструктора у `Animal`. Объяснение на семинаре.
    
</p>    
</details>

<details>
<summary>Почему это не страшно? (но поправить ошибку всё равно желательно)</summary>
<p>
    
Деструкторы Cat && Dog ничего не делает.
    
</p>    
</details>

```c++
void make_it_cry(Animal& animal)
{
    animal.cry();
    animal.cry();
    animal.cry();
}

Animal animal;
make_it_cry(animal);

Dog dog;
make_it_cry(dog);
```

```c++
void make_it_cry_more(Animal animal)
{
    for (int i = 0; i < 100; ++i)
        animal.cry();
}

Animal animal;
make_it_cry_more(animal);

Dog dog;
make_it_cry_more(dog);
```

А что здесь?

<br />

##### Чисто виртуальные функции

По большому счёту, метод `cry()` не имеет смысла для класса `Animal`, т.к. с философской точки зрения непонятно какая реализация у него должна быть. Можно указать, что у виртуальной функции `Animal::cry()` нет реализации. Такие функции называются **чисто виртуальными**.

```c++
class Animal
{
public:
    virtual void cry() = 0;
};

// далее аналогично ...
class Dog : public Animal { ... };
class Cat : public Animal { ... };
```

__Вопрос:__

<details>
<summary>Что ещё принципиально поменялось для класса Animal?</summary>
<p>

Объекты типа `Animal` нельзя создавать.

</p>
</details>

Ещё пример:

```c++
class BridgeBuilder
{
public:
    virtual Bridge make_bridge() = 0;    
};

class StoneBridgeBuilder : public BridgeBuilder
{
public:
    virtual Bridge make_bridge() { return StoneBridge(); }    
};

class WoodBridgeBuilder : public BridgeBuilder
{
public:
    virtual Bridge make_bridge() { return WoodBridge(); }    
};

Landscape make_landscape(BridgeBuilder& bridge_builder)
{
    const auto bridge1 = bridge_builder.make_bridge();
    const auto bridge2 = bridge_builder.make_bridge();
    ...;
}
```

__Вопрос знатокам:__

<details>
<summary>Что-нибудь напоминает?</summary>
<p>

Abstract Factory pattern.

</p>
</details>

С философской точки зрения ООП чисто виртуальные функции лучше отражают ООП-модель программы. Но и обычные виртуальные функции применимы.

__Вопрос:__ можете ли привести пример?

Пример:

```c++
class NetworkSubsystem
{
public:
    virtual void initFromConfig() {
        // initialization code
    }
};

// for special testing
class NetworkSubsystemWithPeriodicDisconnects : public NetworkSubsystem
{
public:
    virtual void initFromConfig() {
        // init core network subsystem
        NetworkSubsystem::initFromConfig();  // TODO: надо не забыть вызвать метод базового класса!
        
        // initialize disconnect settings below
        ...
    }
};
```

<br />

##### virtual / override / final

* `virtual` - сделать метод виртуальным, чтобы наследники могли его переопределять
* `override` - указание компилятору убедиться, что метод виртуальный, если нет - ошибка компиляции
* `final` - запретить наследникам переопределять виртуальный метод

Зачем нужен `override`:

Иерархия сегодня:

```c++
class Animal
{
public:
    virtual void cry() {}
};

class Dog : public Animal
{
public:
    void cry() { std::cout << "augh!"; }
};
```

Иерархия через пару месяцев:

```c++
class Animal
{
public:
    virtual void cry(bool loud) {}
};

class Dog : public Animal
{
public:
    void cry() { std::cout << "augh!"; } // OOOPS, it comiles but is not working as expected anymore
};
```

Как надо было делать иерархию "сегодня":

```c++
class Animal
{
public:
    virtual ~Animal() {}
    virtual void cry() {}    
};

class Dog : public Animal
{
public:
    void cry() override { std::cout << "augh!"; }
};
```

Тогда ошибку поймал бы компилятор, а не пользователь.

<br />

##### vftable

Один из способов реализации механизмов виртуальных функций - компилятор может вписать в класс дополнительное поле - указатель на таблицу виртуальных функций:

```c++
class Animal
{
    std::uint64_t age;
public:
    virtual void cry();
    virtual void jump();
};

class Dog : public Animal
{
    std::uint64_t color;
public:
    void cry() override;
    void jump() override;
};
```

![](vftable.jpg)

Вопросы:

<details>
<summary>sizeof(Animal)</summary>
16
</details>
    
<details>
<summary>sizeof(Dog)</summary>
24
</details>

<br />

##### Множественное наследование, ромбовидное наследование

Обратить внимание на порядок конструирования, вызова деструкторов и способа разрешения конфликтов по именам

```c++
class Animal
{
    ...
};

class JumpingCreature
{
public:
    void jump();
};

class CryingCreature
{
public:
    void cry();    
};


class Cat : public Animal
          , public JumpingCreature
          , public CryingCreature
{
public:
    void play()
    {
        jump();
        jump();
        cry();
        jump();
        
        // явное указание метода,
        //
        // потребуется, если метод jump()
        // есть и у JumpingCreature, и у Animal
        JumpingCreature::jump();
        CryingCreature::cry();
    }
};
```

<img src="classes_diagram_cat.png" height=500px width=500px />

<br />

Граф наследования ацикличен и однонаправлен... но ничто не мешает сделать в нём ромб. Такая конструкция выглядит необычной, но, тем не менее, используется даже в стандартной библиотеке:

![](std-basic_iostream-inheritance.png)

https://en.cppreference.com/w/cpp/io/basic_iostream

<br />

##### Layout класса

Рассмотрим как устроено расположение класса в памяти. Пойдём от простого к сложному.

```c++
struct Point
{
    float x;  // size = 4, alignment = 4
    float y;  // size = 4, alignment = 4
};  // size = 8, alignment = 4
```

![](layout_point.jpg)

```c++
class A
{
    float x;          // size = 4, alignment = 4
    std::uint64_t y;  // size = 8, alignment = 8  
};  // size = ???, alignment = ???
```

![](layout_padding.jpg)

Более подробно про alignment:
https://en.cppreference.com/w/c/language/object

```c++
class A
{
    float x;          // size = 4, alignment = 4
    std::uint64_t y;  // size = 8, alignment = 8
    float z;          // size = 4, alignment = 4
};  // size = ???, alignment = ???
```

![](layout_padding_2.jpg)

```c++
class A
{
    std::uint64_t y;  // size = 8, alignment = 8
    float x;          // size = 4, alignment = 4
    float z;          // size = 4, alignment = 4
};  // size = ???, alignment = ???
```

![](layout_padding_3.jpg)

<br />

Рассмотрим layout класса в памяти в случае наследования:

```c++
struct P
{
    float x;  // size = 4, alignment = 4
    float y;  // size = 4, alignment = 4
};  // size = 8, alignment = 4

struct WP : P
{
    float w;  // size = 4, alignment = 4
};  // size = ???, alignment = ???
```

![](layout_inheritance.jpg)

Рассмотрим layout класса в памяти в случае множественного наследования:

```c++
class CBase1 { ... };
class CBase2 { ... };

class CDerived : public CBase1
               , public CBase2
{ ... };
```

![](layout_multiple_inheritance.jpg)

<br />

Как быть с виртуальными методами и функциями:

```c++
class Base
{
public:
    virtual void say_hello();
    virtual void say_goodbye();
    
private:
    ...
};

class Dervied : public Base
{
public:
    void say_hello() override;
    void say_goodbye() override;
    
private:
    ...
};
```

![](layout_vtable_3.jpg)

Показать как ведёт себя преобразование указателей для такого лаяута (и почему с reinterpert_cast лучше не переборщить).

Открыть на godbolt.org пример и показать как происходит сравнение указателей, закомментировать виртуальные функции - посмотреть что происходит.

```c++
struct Base1
{
    // virtual void say_hello() = 0;
    float x;
};

struct Base2
{
    float x;
};

struct Derived : public Base1
               , public Base2
{
    // virtual void say_hello() = 0;
    float y;
};

bool equal1(Base1* lhs, Derived *rhs)
{
    return lhs == rhs;
}

bool equal2(Base2* lhs, Derived *rhs)
{
    return lhs == rhs;
}
```

__Вопрос на понимание:__ каков layout класса при ромбовидном наследовании?

__Вопрос для продвинутых:__ как его оптимизировать?

<br />

##### Состояние класса, const + mutable

Что считать состоянием класса?
* значения всех его полей
* логическое состояние класса (набор элементов в векторе, а не детали типа `capacity`)

С точки зрения компилятора - набор всех его полей, не помеченных словом `mutable`

```c++
class Animal
{
private:
    std::string name;
    unsigned age;
    
public:
    // аттрибут const означает, что метод не будет
    // менять полей класса (и компилятор это проверит!)
    //
    // внутри const-методов поля имеют тип:
    // const std::string name;
    // const unsigned age;
    bool is_too_young() const
    {
        return age <= 3;
    }
    
    // метод не меняет полей класса, но программист
    // забыл поставить const
    void say_hello()
    {
        std::cout << "hello";
    }
    
    // метод не меняет полей класса
    void say_hello_twice() const
    {
        say_hello();  // COMPILE-TIME ERROR
        say_hello();
        
        name = "unk";  // COMPILE-TIME ERROR
    }
    
    // отсутствие const - метод МОЖЕТ менять поля
    void happy_birthday()
    {
        age += 1; // OK
    }   
};
```

Рассмотрим учебный пример - отображение, которое хранит ответ на последний запрос как кеширование.

```c++
class Resources
{
public:
    void add(int id, const std::string& resource);
    
    const std::string& get(int id) const;
    
private:
    // хранилище ресурсов
    std::map<int, std::string> id_to_resource;
    
    // закешированный последний запрошенный ресурс
    mutable int last_id = -1;
    mutable std::string last_resource;
};

void Resources::add(const int id, const std::string& resource)
{
    id_to_resource[id] = resource;
}

const std::string& Resources::get(const int id) const
{
    if (id == last_id)
        return last_resource;
    
    auto it = id_to_resource.find(id);  // |find| - const-метод
    
    last_resource = it->second;  // OK, т.к. mutable
    last_id = id;                // OK, т.к. mutable
    
    return last_resource;
}
```

Примечание: это исключительно ученический пример кеширования для демонстрации понятий `mutable` и состояния объекта, никогда так не реализуйте кеширование.

<br />

##### Операторы

https://en.cppreference.com/w/cpp/language/operators

В виде свободных функций

```c++
struct Vector3D
{
    double x;
    double y;
    double z;
};

// v1 + v2
Vector3D operator + (const Vector3D& l, const Vector3D& r)
{
    return {l.x + r.x, l.y + r.y, l.z + r.z};
}

// v1 * v2
double operator * (const Vector3D& l, const Vector3D& r)
{
    return l.x * r.x + l.y * r.y + l.z * r.z;
}

// v1 * a
Vector3D operator *(const Vector3D& v, const double a)
{
    return {v.x * a, v.y * a, v.z * a};
}

// a * v1
Vector3D operator *(const double a, const Vector3D& v)
{
    return v * a;
}

// v1 / a
Vector3D operator *(const Vector3D& v, const double a)
{
    return v * (1. / a);
}
```

В виде членов класса и свободных функций-друзей:

```c++
class Animal
{
private:
    std::string name;
    unsigned age;
    
public:
    // Animal a;
    // ++a;
    Animal& operator++()
    {
        ++age;
        return *this;
    }
    
    // Animal a;
    // a++;
    Animal operator++(int)
    {
        Animal tmp = *this;
        operator++();
        return tmp;
    }

    // Animal a;
    // std::cout << a;
    friend std::ostream& operator <<(std::ostream& os, const Animal& animal);
};

std::ostream& operator <<(std::ostream& os, const Animal& animal)
{
    return os << animal.name << "," << animal.age;
}
```

Разница:
https://www.quora.com/What-is-the-difference-between-defining-an-overloaded-operator-outside-the-class-and-defining-it-inside-the-class/answer/Sergey-Zubkov-1

<br />

##### static-поля класса

Класс может иметь `static`-поля и `static`-методы:

`static`-поля можно рассматривать как глобальные константы / переменные:

```c++

// Animal.h

class Animal
{
public:
    Animal(const std::string& a_name, unsigned a_age);
    ...
    
    // declaration
    static const std::string name_of_unknown;    
};

// Animal.cpp

// definition
const std::string Animal::name_of_unknown = "UNK";

// main.cpp

// usage:
void func()
{
    std::cout << "name of uknown animal is: " << Animal::name_of_unknown << std::endl;
    
    Animal animal(Animal::name_of_unknown, 12);
}
```

На `static`-поля распространяются модификаторы области доступа `private`/`protected`/`public`:
* В случае `public` `static`-поле ведёт себя как обычная глобальная константа
* В случае `private` только методы класса имеют доступ до этой константы

__Вопрос__: 

<details>
<summary>Зачем могут быть полезны `static`-поля класса</summary>
<p>
   
* Классоспецифичные константы.
* Классоспецифичные глобальные переменные (по возможности избегать).
   
</p>
</details>


<details>
<summary>Ещё примеры?</summary>
<p>
   
* Класс DateTime. Можно в `public` вынести константы minute, second, hour ...
* Ещё?
   
</p>
</details>

<br />

##### static-методы класса

`static`-метод класса можно рассматривать как свободную friend-функцию:

```c++
class Animal
{
    int age = 0;

public:
    static void grow_up(int count, Animal& an)
    {
        an.age += count;
    }

    int get_age() const { return age; }
};

int main()
{
    Animal an;
    Animal::grow_up(4, an);  // TODO: обратить внимание на вызов
    std::cout << an.get_age() << std::endl;
    return 0;
}
```

`static`-метод - обычная friend-функция, находящаяся в "пространстве имён" класса.

**Для вызова static-метода _НЕ_ нужен объект класса**

Пример использования `static`-метода:

```c++
class House
{
    Floor floor;
    Roof roof;
    std::vector<Wall> walls;

public:
    House(Floor f, Roof r, std::vector<Wall> walls);
    
    static House make(const Config& config)
    {
        auto floor = Floor::make(config["floor"]);
        auto root = Roof::make(config["roof"]);
        auto walls = ...;        
        return House(floor, roof, walls);
    }    
};
```

__Вопрос__: что-нибудь напоминает?

__Вопрос__: когда оправдано использовать static-методы?

<br />

__Замечания по лекции:__
* Расширить материал по операторам, добавить примеры, добавить более адекватные примеры, сейчас материал скомкан